<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» на BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**Используемые библиотеки**

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import re 
import nltk
from tqdm import notebook
import torch
import transformers 
import matplotlib.pyplot as plt
%matplotlib inline

from wordcloud import WordCloud
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier

In [ ]:
def preprocess_text(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

In [ ]:
RANDOM_STATE=42

## Подготовка

In [ ]:
try:
    data = pd.read_csv('toxic_comments.csv', index_col=0) 
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=0)

In [ ]:
data.info()

Очистим текст от лишних символов

In [ ]:
data['text'] = data['text'].apply(preprocess_text)

Проверим данные на дубликаты

In [ ]:
data.duplicated().sum()

Удалим дубликаты

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

In [ ]:
data.duplicated().sum()

Дубликатов нет

In [ ]:
text_no_toxic = " ".join(data.query("toxic==0")['text'])
wordcloud_no_toxic = WordCloud().generate(text_no_toxic)
plt.imshow(wordcloud_no_toxic)

In [ ]:
text_toxic = " ".join(data.query("toxic==1")['text'])
wordcloud_toxic = WordCloud().generate(text_toxic)
plt.imshow(wordcloud_toxic)

Проверим данные на дисбаланс классов

In [ ]:
data.hist()

Имеется явный дисбаланс. Выровняем классы на этапе обучения моделей (разделение данных).

Перепроверим

Для ускорения векторизации уменьшим количество строк до 2000.

In [ ]:
data = data.sample(n=2000, random_state=RANDOM_STATE)

Проведем эмбеддинг BERT

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained("unitary/toxic-bert")
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenized = data['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
model = transformers.BertModel.from_pretrained("unitary/toxic-bert", num_labels = 2)
#model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

In [ ]:
batch_size = 1
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

## Обучение

Разделим таблицу.

In [ ]:
X = np.concatenate(embeddings)

In [ ]:
y = data['toxic']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size = .25, 
    stratify = y,
    random_state=RANDOM_STATE)

In [ ]:
X_test.shape

In [ ]:
features_down = pd.DataFrame(X_train).reset_index(drop=True)
target_down = pd.Series(y_train).reset_index(drop=True)
features_zeros = features_down[target_down == 0]
features_ones = features_down[target_down == 1]
target_zeros = target_down[target_down == 0]
target_ones = target_down[target_down == 1]
features_downsampled = pd.concat(
    [features_zeros.sample(frac=0.15, random_state=RANDOM_STATE)] + [features_ones]
)
target_downsampled = pd.concat(
    [target_zeros.sample(frac=0.15, random_state=RANDOM_STATE)] + [target_ones]
)
features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=RANDOM_STATE
)



In [ ]:
features_up = pd.DataFrame(X_train).reset_index(drop=True)
target_up = pd.Series(y_train).reset_index(drop=True)
features_zeros = features_up[target_up == 0]
features_ones = features_up[target_up == 1]
target_zeros = target_up[target_up == 0]
target_ones = target_up[target_up == 1]

features_upsampled = pd.concat(
    [features_zeros.sample(frac=0.10, random_state=RANDOM_STATE)] + [features_ones] * 3
)
target_upsampled = pd.concat(
    [target_zeros.sample(frac=0.10, random_state=RANDOM_STATE)] + [target_ones] * 3
)
    
features_upsampled, target_upsampled = shuffle(
features_upsampled, target_upsampled, random_state=RANDOM_STATE)

In [ ]:
smo_tek = SMOTE(random_state=RANDOM_STATE, k_neighbors=10)
X_smotek, y_smotek = smo_tek.fit_resample(X_train, y_train)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

sampler = RandomOverSampler(random_state=RANDOM_STATE)
X_overresample, y_overresample = sampler.fit_resample(X_train, y_train) 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_underresample, y_underresample = sampler.fit_resample(X_train, y_train) 

In [ ]:
y_smotek

Обучим модели

### LogisticRegression

In [ ]:
params = {
            'C': range(1,10),
            'penalty': [None,'l1','l2'],
             'solver' : ['liblinear','lbfgs']
          } 
            

lr_clf = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000, class_weight='balanced') 
lr_model = GridSearchCV(lr_clf, param_grid=params, scoring='f1', cv = 4, n_jobs=1) 
lr_model.fit(X_train, y_train)
lr_model.best_score_ 

### RandomForestClassifier

In [ ]:
params = {
             'n_estimators': [10, 50, 100, 300],
             'max_depth': [1, 4, 9, 50]
            } 

rfr = RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced') 
rfr_model = GridSearchCV(rfr, param_grid=params, scoring='f1', cv = 4, n_jobs=1) 
rfr_model.fit(X_train, y_train)

In [ ]:
rfr_model.best_score_

### LGBMClassifier

In [ ]:
param_dist = {"max_depth": [1, 4, 9, 50],
              "num_leaves": [100],
              "n_estimators": [10, 50, 100, 300],
             }

lgr = LGBMClassifier(random_state=RANDOM_STATE, class_weight='balanced')
LGBM_model = GridSearchCV(lgr, param_grid=param_dist, cv = 4, n_jobs=1, scoring='f1', verbose=5)
LGBM_model.fit(X_train, y_train)

In [ ]:
LGBM_model.best_score_

### LinearSVC

In [ ]:
params = {
    'C': [0.1, 1, 10, 50]
          }


svc = LinearSVC(random_state=RANDOM_STATE, class_weight='balanced') 
svc_model = GridSearchCV(svc, param_grid=params, scoring='f1', cv = 4, n_jobs=1) 
svc_model.fit(X_train, y_train)

In [ ]:
svc_model.best_score_

### CatBoostClassifier

In [ ]:
cbc = CatBoostClassifier(iterations = 200, learning_rate = 0.5, verbose = 20, random_state = RANDOM_STATE)
cbc_model = GridSearchCV(svc, param_grid=params, scoring='f1', cv = 4, n_jobs=1) 
cbc_model.fit(X_train, y_train)

In [ ]:
cbc_model.best_score_

F1 лучшей модели:

In [ ]:
f1_score(y_test, rfr_model.predict(X_test))

## Выводы

В ходе проекта для интернет-магазина "Викишоп" была обучена модель для поиска токсичных комментариев при помощи BERT. Данные были подготовлены, Среди всех моделей лучшей оказалась RandomForestClassifie.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны

In [ ]:
print("Для проверки кода на работоспособность")